In [2]:
# Import dependencies
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import requests
import json
import pprint
from uszipcode import SearchEngine
from config import api_key
from census import Census

In [13]:
# Read in data from csv file
crime_df = pd.read_csv("chicago_crimes_all_cleaned.csv")

# Remove District 31 as it has a very low count
crime_df = crime_df[crime_df["District"] != 31]
crime_df = crime_df[["Case Number", "Date", "Block", "Primary Type", "Description", "District", "Ward",
                                     "Year", "Latitude", "Longitude", "Location", "Population", "Zipcode"]]
crime_df

,Case Number,Date,Block,Primary Type,Description,District,Ward,Year,Latitude,Longitude,Location,Population,Zipcode
0,JC104662,12/31/2018 11:59:00 PM,112XX S SACRAMENTO AVE,MINOR PROPERTY CRIME,TO VEHICLE,22,19.0,2018,41.689079,-87.696064,"(41.689078832, -87.696064026)",28804.0,60655.0
1,JC100043,12/31/2018 11:57:00 PM,084XX S SANGAMON ST,MINOR PROPERTY CRIME,TO PROPERTY,6,21.0,2018,41.740521,-87.647391,"(41.740520866, -87.647390719)",68096.0,60620.0
2,JC100006,12/31/2018 11:56:00 PM,018XX S ALLPORT ST,ASSAULT & BATTERY,AGG: HANDS/FIST/FEET NO/MINOR INJURY,12,25.0,2018,41.857068,-87.657625,"(41.857068095, -87.657625201)",79205.0,60608.0
3,JC100031,12/31/2018 11:55:00 PM,078XX S SANGAMON ST,ASSAULT & BATTERY,DOMESTIC BATTERY SIMPLE,6,17.0,2018,41.751914,-87.647717,"(41.75191443, -87.647716532)",68096.0,60620.0
4,JC100026,12/31/2018 11:49:00 PM,052XX W GLADYS AVE,ASSAULT & BATTERY,AGGRAVATED - HANDGUN,15,29.0,2018,41.875684,-87.760479,"(41.87568438, -87.760479356)",47712.0,60644.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1145665,JF420379,9/16/2022 13:48,026XX N NARRAGANSETT AVE,THEFT (ALL TYPES),RETAIL THEFT,25,36.0,2022,41.928077,-87.785606,"(41.928076536, -87.785606186)",41309.0,60707.0
1145666,JF420478,9/1/2022 5:00,005XX W SURF ST,OTHER MINOR CRIMES,HARASSMENT BY TELEPHONE,19,44.0,2022,41.934305,-87.641485,"(41.934304581, -87.641484982)",72113.0,60657.0
1145667,JF420319,7/8/2022 0:00,114XX S PRAIRIE AVE,THEFT (ALL TYPES),FRAUD OR CONFIDENCE GAME,5,9.0,2022,41.685544,-87.616813,"(41.685543881, -87.616812541)",64489.0,60628.0
1145668,JF420102,9/27/2022 11:00,023XX E 70TH ST,THEFT (ALL TYPES),OVER $500,3,5.0,2022,41.768068,-87.567453,"(41.768068052, -87.567452932)",48973.0,60649.0


In [14]:
# Create a SearchEngine object 
search = SearchEngine()

In [15]:
# Define a function to get zip code from latitude and longitude
def fill_zip_code(row):
    # Use uszipcode's search by coordinates function, and set a small radius since we're only looking at Chicago
    result = search.by_coordinates(row['Latitude'], row['Longitude'], radius = 5)
    # If the result of the search is not empty
    if result:
        # Return the first element (zipcode) 
        return result[0].zipcode
    else:
        # If the result doesn't have a zipcode to pull, don't return anything
        return None

In [ ]:
# Run the latitude and longitude through the function to get the zip code. Note: this takes an extremely long amount of time.
crime_df["Zipcode"] = crime_df.apply(fill_zip_code, axis=1)



In [16]:
# Check to make sure zip codes were filled
crime_df

,Case Number,Date,Block,Primary Type,Description,District,Ward,Year,Latitude,Longitude,Location,Population,Zipcode
0,JC104662,12/31/2018 11:59:00 PM,112XX S SACRAMENTO AVE,MINOR PROPERTY CRIME,TO VEHICLE,22,19.0,2018,41.689079,-87.696064,"(41.689078832, -87.696064026)",28804.0,60655.0
1,JC100043,12/31/2018 11:57:00 PM,084XX S SANGAMON ST,MINOR PROPERTY CRIME,TO PROPERTY,6,21.0,2018,41.740521,-87.647391,"(41.740520866, -87.647390719)",68096.0,60620.0
2,JC100006,12/31/2018 11:56:00 PM,018XX S ALLPORT ST,ASSAULT & BATTERY,AGG: HANDS/FIST/FEET NO/MINOR INJURY,12,25.0,2018,41.857068,-87.657625,"(41.857068095, -87.657625201)",79205.0,60608.0
3,JC100031,12/31/2018 11:55:00 PM,078XX S SANGAMON ST,ASSAULT & BATTERY,DOMESTIC BATTERY SIMPLE,6,17.0,2018,41.751914,-87.647717,"(41.75191443, -87.647716532)",68096.0,60620.0
4,JC100026,12/31/2018 11:49:00 PM,052XX W GLADYS AVE,ASSAULT & BATTERY,AGGRAVATED - HANDGUN,15,29.0,2018,41.875684,-87.760479,"(41.87568438, -87.760479356)",47712.0,60644.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1145665,JF420379,9/16/2022 13:48,026XX N NARRAGANSETT AVE,THEFT (ALL TYPES),RETAIL THEFT,25,36.0,2022,41.928077,-87.785606,"(41.928076536, -87.785606186)",41309.0,60707.0
1145666,JF420478,9/1/2022 5:00,005XX W SURF ST,OTHER MINOR CRIMES,HARASSMENT BY TELEPHONE,19,44.0,2022,41.934305,-87.641485,"(41.934304581, -87.641484982)",72113.0,60657.0
1145667,JF420319,7/8/2022 0:00,114XX S PRAIRIE AVE,THEFT (ALL TYPES),FRAUD OR CONFIDENCE GAME,5,9.0,2022,41.685544,-87.616813,"(41.685543881, -87.616812541)",64489.0,60628.0
1145668,JF420102,9/27/2022 11:00,023XX E 70TH ST,THEFT (ALL TYPES),OVER $500,3,5.0,2022,41.768068,-87.567453,"(41.768068052, -87.567452932)",48973.0,60649.0


In [17]:
# Change Zip code from an object to a float for merging purposes
crime_df["Zipcode"] = crime_df["Zipcode"].astype(float)

In [ ]:
# Save the file to .csv
crime_df.to_csv("zip_codes_added.csv", index = False)

In [19]:
# Check the amount of null values in the zip code column
print(f"The number of null values is: {crime_df.isna().sum()}")

The number of null values is: Case Number     0
Date            0
Block           0
Primary Type    0
Description     0
District        0
Ward            0
Year            0
Latitude        0
Longitude       0
Location        0
Population      0
Zipcode         0
dtype: int64


In [6]:
# Read in "zip_codes_added" to not run the crime_df["Zipcode"] above again. **Run from here to save time**
crime_df = pd.read_csv("zip_codes_added.csv")

In [7]:
# Read in the census data for 2018-2021. Note that we will have to get the 2022 census popultation data from another source (the census API)
pop_counts = pd.read_csv("Chicago_Population_Counts.csv")
pop_counts

,Geography Type,Year,Zipcode,Population - Total,Population - Age 0-17,Population - Age 18-29,Population - Age 30-39,Population - Age 40-49,Population - Age 50-59,Population - Age 60-69,...,Population - Age 18+,Population - Age 65+,Population - Female,Population - Male,Population - Latinx,Population - Asian Non-Latinx,Population - Black Non-Latinx,Population - White Non-Latinx,Population - Other Race Non-Latinx,Record ID
0,ZIP Code,2018,60601,14675,820,4606,2792,2190,1333,1340,...,13855,2075,7484,7191,1274,NaN,NaN,9677,NaN,ZIP_Code-60601-2018
1,ZIP Code,2018,60602,1244,149,435,462,135,53,10,...,1095,5,551,693,81,NaN,NaN,788,NaN,ZIP_Code-60602-2018
2,ZIP Code,2018,60603,1174,56,561,101,97,197,97,...,1118,112,601,573,115,NaN,NaN,707,NaN,ZIP_Code-60603-2018
3,ZIP Code,2018,60604,782,38,303,104,51,101,130,...,744,93,413,369,34,NaN,NaN,479,NaN,ZIP_Code-60604-2018
4,ZIP Code,2018,60605,27519,2260,8103,6316,3567,3344,2455,...,25259,2569,14381,13138,1608,NaN,NaN,15781,NaN,ZIP_Code-60605-2018
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
226,Zip Code,2021,60626,50295,8007,11937,8965,6995,6738,4360,...,42288,5098,25318,24977,9694,2724.0,12646.0,23029,2202.0,ZIP_CODE-60626-2021
227,Zip Code,2021,60628,64489,13734,11638,7491,7593,9066,6930,...,50755,10940,34961,29528,2775,79.0,59168.0,1462,1005.0,ZIP_CODE-60628-2021
228,Zip Code,2021,60629,107487,29283,20690,13227,14954,11096,8187,...,78204,13746,53753,53734,79111,595.0,18878.0,7650,1253.0,ZIP_CODE-60629-2021
229,Zip Code,2021,60630,55591,11093,7053,8926,7337,7479,7975,...,44498,9658,27787,27804,14180,7122.0,1007.0,31325,1957.0,ZIP_CODE-60630-2021


In [9]:
# Check to see how many null values we have
pop_counts.isna().sum()

Geography Type                         0
Year                                   0
Zipcode                                0
Population - Total                     0
Population - Age 0-17                  0
Population - Age 18-29                 0
Population - Age 30-39                 0
Population - Age 40-49                 0
Population - Age 50-59                 0
Population - Age 60-69                 0
Population - Age 70-79                 0
Population - Age 80+                   0
Population - Age 0-4                   0
Population - Age 5-11                 59
Population - Age 12-17                59
Population - Age 5+                    0
Population - Age 18+                   0
Population - Age 65+                   0
Population - Female                    0
Population - Male                      0
Population - Latinx                    0
Population - Asian Non-Latinx         59
Population - Black Non-Latinx         59
Population - White Non-Latinx          0
Population - Oth

In [10]:
# Merge crime DataFrame with population counts by zipcode
merged_df = crime_df.merge(pop_counts, on = ["Zipcode", "Year"], how = "left")
merged_df

,Case Number,Date,Block,Primary Type,Description,District,Ward,Year,Latitude,Longitude,...,Population - Age 18+,Population - Age 65+,Population - Female,Population - Male,Population - Latinx,Population - Asian Non-Latinx,Population - Black Non-Latinx,Population - White Non-Latinx,Population - Other Race Non-Latinx,Record ID
0,JC104662,12/31/2018 11:59:00 PM,112XX S SACRAMENTO AVE,MINOR PROPERTY CRIME,TO VEHICLE,22,19.0,2018,41.689079,-87.696064,...,21789.0,3731.0,14928.0,13876.0,2390.0,NaN,NaN,23795.0,NaN,ZIP_Code-60655-2018
1,JC100043,12/31/2018 11:57:00 PM,084XX S SANGAMON ST,MINOR PROPERTY CRIME,TO PROPERTY,6,21.0,2018,41.740521,-87.647391,...,53412.0,11842.0,38095.0,30001.0,911.0,NaN,NaN,755.0,NaN,ZIP_Code-60620-2018
2,JC100006,12/31/2018 11:56:00 PM,018XX S ALLPORT ST,ASSAULT & BATTERY,AGG: HANDS/FIST/FEET NO/MINOR INJURY,12,25.0,2018,41.857068,-87.657625,...,64295.0,7852.0,35360.0,43845.0,40146.0,NaN,NaN,15148.0,NaN,ZIP_Code-60608-2018
3,JC100031,12/31/2018 11:55:00 PM,078XX S SANGAMON ST,ASSAULT & BATTERY,DOMESTIC BATTERY SIMPLE,6,17.0,2018,41.751914,-87.647717,...,53412.0,11842.0,38095.0,30001.0,911.0,NaN,NaN,755.0,NaN,ZIP_Code-60620-2018
4,JC100026,12/31/2018 11:49:00 PM,052XX W GLADYS AVE,ASSAULT & BATTERY,AGGRAVATED - HANDGUN,15,29.0,2018,41.875684,-87.760479,...,35376.0,6201.0,25669.0,22043.0,3068.0,NaN,NaN,1392.0,NaN,ZIP_Code-60644-2018
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1145665,JF420379,9/16/2022 13:48,026XX N NARRAGANSETT AVE,THEFT (ALL TYPES),RETAIL THEFT,25,36.0,2022,41.928077,-87.785606,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1145666,JF420478,9/1/2022 5:00,005XX W SURF ST,OTHER MINOR CRIMES,HARASSMENT BY TELEPHONE,19,44.0,2022,41.934305,-87.641485,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1145667,JF420319,7/8/2022 0:00,114XX S PRAIRIE AVE,THEFT (ALL TYPES),FRAUD OR CONFIDENCE GAME,5,9.0,2022,41.685544,-87.616813,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1145668,JF420102,9/27/2022 11:00,023XX E 70TH ST,THEFT (ALL TYPES),OVER $500,3,5.0,2022,41.768068,-87.567453,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [11]:
# See what columns we have
merged_df.columns

Index(['Case Number', 'Date', 'Block', 'Primary Type', 'Description',
       'District', 'Ward', 'Year', 'Latitude', 'Longitude', 'Location',
       'Population', 'Zipcode', 'Geography Type', 'Population - Total',
       'Population - Age 0-17', 'Population - Age 18-29',
       'Population - Age 30-39', 'Population - Age 40-49',
       'Population - Age 50-59', 'Population - Age 60-69',
       'Population - Age 70-79', 'Population - Age 80+',
       'Population - Age 0-4', 'Population - Age 5-11',
       'Population - Age 12-17', 'Population - Age 5+', 'Population - Age 18+',
       'Population - Age 65+', 'Population - Female', 'Population - Male',
       'Population - Latinx', 'Population - Asian Non-Latinx',
       'Population - Black Non-Latinx', 'Population - White Non-Latinx',
       'Population - Other Race Non-Latinx', 'Record ID'],
      dtype='object')

In [12]:
# Clean columns to the ones we want
crime_cleaned_df = merged_df[["Case Number", "Date", "Block", "Primary Type", "Description", "District", "Ward",
                                     "Year", "Latitude", "Longitude", "Location", "Population - Total", "Zipcode"]]
crime_cleaned_df

,Case Number,Date,Block,Primary Type,Description,District,Ward,Year,Latitude,Longitude,Location,Population - Total,Zipcode
0,JC104662,12/31/2018 11:59:00 PM,112XX S SACRAMENTO AVE,MINOR PROPERTY CRIME,TO VEHICLE,22,19.0,2018,41.689079,-87.696064,"(41.689078832, -87.696064026)",28804.0,60655.0
1,JC100043,12/31/2018 11:57:00 PM,084XX S SANGAMON ST,MINOR PROPERTY CRIME,TO PROPERTY,6,21.0,2018,41.740521,-87.647391,"(41.740520866, -87.647390719)",68096.0,60620.0
2,JC100006,12/31/2018 11:56:00 PM,018XX S ALLPORT ST,ASSAULT & BATTERY,AGG: HANDS/FIST/FEET NO/MINOR INJURY,12,25.0,2018,41.857068,-87.657625,"(41.857068095, -87.657625201)",79205.0,60608.0
3,JC100031,12/31/2018 11:55:00 PM,078XX S SANGAMON ST,ASSAULT & BATTERY,DOMESTIC BATTERY SIMPLE,6,17.0,2018,41.751914,-87.647717,"(41.75191443, -87.647716532)",68096.0,60620.0
4,JC100026,12/31/2018 11:49:00 PM,052XX W GLADYS AVE,ASSAULT & BATTERY,AGGRAVATED - HANDGUN,15,29.0,2018,41.875684,-87.760479,"(41.87568438, -87.760479356)",47712.0,60644.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1145665,JF420379,9/16/2022 13:48,026XX N NARRAGANSETT AVE,THEFT (ALL TYPES),RETAIL THEFT,25,36.0,2022,41.928077,-87.785606,"(41.928076536, -87.785606186)",NaN,60707.0
1145666,JF420478,9/1/2022 5:00,005XX W SURF ST,OTHER MINOR CRIMES,HARASSMENT BY TELEPHONE,19,44.0,2022,41.934305,-87.641485,"(41.934304581, -87.641484982)",NaN,60657.0
1145667,JF420319,7/8/2022 0:00,114XX S PRAIRIE AVE,THEFT (ALL TYPES),FRAUD OR CONFIDENCE GAME,5,9.0,2022,41.685544,-87.616813,"(41.685543881, -87.616812541)",NaN,60628.0
1145668,JF420102,9/27/2022 11:00,023XX E 70TH ST,THEFT (ALL TYPES),OVER $500,3,5.0,2022,41.768068,-87.567453,"(41.768068052, -87.567452932)",NaN,60649.0


In [13]:
# Change the name of "Population - Total" so it's easier to merge
crime_cleaned_df = crime_cleaned_df.rename(columns={"Population - Total" : "Population"})
crime_cleaned_df

,Case Number,Date,Block,Primary Type,Description,District,Ward,Year,Latitude,Longitude,Location,Population,Zipcode
0,JC104662,12/31/2018 11:59:00 PM,112XX S SACRAMENTO AVE,MINOR PROPERTY CRIME,TO VEHICLE,22,19.0,2018,41.689079,-87.696064,"(41.689078832, -87.696064026)",28804.0,60655.0
1,JC100043,12/31/2018 11:57:00 PM,084XX S SANGAMON ST,MINOR PROPERTY CRIME,TO PROPERTY,6,21.0,2018,41.740521,-87.647391,"(41.740520866, -87.647390719)",68096.0,60620.0
2,JC100006,12/31/2018 11:56:00 PM,018XX S ALLPORT ST,ASSAULT & BATTERY,AGG: HANDS/FIST/FEET NO/MINOR INJURY,12,25.0,2018,41.857068,-87.657625,"(41.857068095, -87.657625201)",79205.0,60608.0
3,JC100031,12/31/2018 11:55:00 PM,078XX S SANGAMON ST,ASSAULT & BATTERY,DOMESTIC BATTERY SIMPLE,6,17.0,2018,41.751914,-87.647717,"(41.75191443, -87.647716532)",68096.0,60620.0
4,JC100026,12/31/2018 11:49:00 PM,052XX W GLADYS AVE,ASSAULT & BATTERY,AGGRAVATED - HANDGUN,15,29.0,2018,41.875684,-87.760479,"(41.87568438, -87.760479356)",47712.0,60644.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1145665,JF420379,9/16/2022 13:48,026XX N NARRAGANSETT AVE,THEFT (ALL TYPES),RETAIL THEFT,25,36.0,2022,41.928077,-87.785606,"(41.928076536, -87.785606186)",NaN,60707.0
1145666,JF420478,9/1/2022 5:00,005XX W SURF ST,OTHER MINOR CRIMES,HARASSMENT BY TELEPHONE,19,44.0,2022,41.934305,-87.641485,"(41.934304581, -87.641484982)",NaN,60657.0
1145667,JF420319,7/8/2022 0:00,114XX S PRAIRIE AVE,THEFT (ALL TYPES),FRAUD OR CONFIDENCE GAME,5,9.0,2022,41.685544,-87.616813,"(41.685543881, -87.616812541)",NaN,60628.0
1145668,JF420102,9/27/2022 11:00,023XX E 70TH ST,THEFT (ALL TYPES),OVER $500,3,5.0,2022,41.768068,-87.567453,"(41.768068052, -87.567452932)",NaN,60649.0


In [14]:
# Check the amount of null values in the merged data frame.  
print(f"The number of null values is: {crime_cleaned_df.isna().sum()}")

The number of null values is: Case Number          0
Date                 0
Block                0
Primary Type         0
Description          0
District             0
Ward                 0
Year                 0
Latitude             0
Longitude            0
Location             0
Population      252814
Zipcode              0
dtype: int64


In [15]:
# See the year the null values are coming from
null_population = crime_cleaned_df.loc[crime_cleaned_df["Population"].isnull()]
null_population

,Case Number,Date,Block,Primary Type,Description,District,Ward,Year,Latitude,Longitude,Location,Population,Zipcode
892856,JC101958,12/31/2018 11:00:00 PM,100XX W OHARE ST,THEFT (ALL TYPES),CREDIT CARD FRAUD,16,41.0,2018,41.976290,-87.905227,"(41.976290414, -87.905227221)",NaN,60106.0
892857,JB574311,12/31/2018 09:30:00 PM,042XX W 31ST ST,ASSAULT & BATTERY,SIMPLE,10,22.0,2018,41.836806,-87.730955,"(41.836806313, -87.73095549)",NaN,60804.0
892858,JC101161,12/31/2018 09:00:00 PM,030XX S KOLIN AVE,MINOR PROPERTY CRIME,TO VEHICLE,10,22.0,2018,41.837771,-87.732880,"(41.837771398, -87.732880358)",NaN,60804.0
892859,JB574199,12/31/2018 06:20:00 PM,100XX W OHARE ST,THEFT (ALL TYPES),CREDIT CARD FRAUD,16,41.0,2018,41.976290,-87.905227,"(41.976290414, -87.905227221)",NaN,60106.0
892860,JB573087,12/30/2018 04:00:00 PM,073XX W LUNT AVE,THEFT (ALL TYPES),FROM BUILDING,16,41.0,2018,42.007986,-87.810471,"(42.007985889, -87.810471311)",NaN,60714.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1145665,JF420379,9/16/2022 13:48,026XX N NARRAGANSETT AVE,THEFT (ALL TYPES),RETAIL THEFT,25,36.0,2022,41.928077,-87.785606,"(41.928076536, -87.785606186)",NaN,60707.0
1145666,JF420478,9/1/2022 5:00,005XX W SURF ST,OTHER MINOR CRIMES,HARASSMENT BY TELEPHONE,19,44.0,2022,41.934305,-87.641485,"(41.934304581, -87.641484982)",NaN,60657.0
1145667,JF420319,7/8/2022 0:00,114XX S PRAIRIE AVE,THEFT (ALL TYPES),FRAUD OR CONFIDENCE GAME,5,9.0,2022,41.685544,-87.616813,"(41.685543881, -87.616812541)",NaN,60628.0
1145668,JF420102,9/27/2022 11:00,023XX E 70TH ST,THEFT (ALL TYPES),OVER $500,3,5.0,2022,41.768068,-87.567453,"(41.768068052, -87.567452932)",NaN,60649.0


In [16]:
# Count the amount of null values per year. Note we have not added population for 2022 yet
year_null = null_population["Year"].value_counts()
year_null

2022    228855
2020     18970
2019      1869
2018      1689
2021      1431
Name: Year, dtype: int64

In [17]:
# Check the data types of the merged data frame
col_types = crime_cleaned_df.dtypes
print(col_types)

Case Number      object
Date             object
Block            object
Primary Type     object
Description      object
District          int64
Ward            float64
Year              int64
Latitude        float64
Longitude       float64
Location         object
Population      float64
Zipcode         float64
dtype: object


In [19]:
# Create an instance of the Census library
c = Census(
    api_key,
    year = 2022
)


In [20]:
# Run Census Search to retrieve population data on all zip codes (ACS5 Census). We can then match them up to the crime table later.
census_data = c.acs5.get(
    (
        "NAME",
        # Estimate of the total population
        "B01003_001E"
    ),
    {'for': 'zip code tabulation area:*'}
)

# Convert to DataFrame
census_pd = pd.DataFrame(census_data)

# Column renaming
census_pd = census_pd.rename(
    columns = {
        "B01003_001E": "Population",
        "zip code tabulation area": "Zipcode"
    }
)

# Configure the final DataFrame
census_pd = census_pd[
    [
        "Population",
        "Zipcode" 
    ]
]

# Display DataFrame length and sample data
print(f"Number of rows in the DataFrame: {len(census_pd)}")
census_pd.head()

Number of rows in the DataFrame: 33774


,Population,Zipcode
0,17126.0,00601
1,37895.0,00602
2,49136.0,00603
3,5751.0,00606
4,26153.0,00610


In [21]:
# Change zipcode from object to integer
census_pd["Zipcode"] = census_pd["Zipcode"].astype(int)


In [22]:
# This fills in filling in missing null values with the census data
cen_merge_df = pd.merge(crime_cleaned_df[crime_cleaned_df['Population'].isnull()], census_pd, on='Zipcode', how = "left")
cen_merge_df

,Case Number,Date,Block,Primary Type,Description,District,Ward,Year,Latitude,Longitude,Location,Population_x,Zipcode,Population_y
0,JC101958,12/31/2018 11:00:00 PM,100XX W OHARE ST,THEFT (ALL TYPES),CREDIT CARD FRAUD,16,41.0,2018,41.976290,-87.905227,"(41.976290414, -87.905227221)",NaN,60106.0,20694.0
1,JB574311,12/31/2018 09:30:00 PM,042XX W 31ST ST,ASSAULT & BATTERY,SIMPLE,10,22.0,2018,41.836806,-87.730955,"(41.836806313, -87.73095549)",NaN,60804.0,84905.0
2,JC101161,12/31/2018 09:00:00 PM,030XX S KOLIN AVE,MINOR PROPERTY CRIME,TO VEHICLE,10,22.0,2018,41.837771,-87.732880,"(41.837771398, -87.732880358)",NaN,60804.0,84905.0
3,JB574199,12/31/2018 06:20:00 PM,100XX W OHARE ST,THEFT (ALL TYPES),CREDIT CARD FRAUD,16,41.0,2018,41.976290,-87.905227,"(41.976290414, -87.905227221)",NaN,60106.0,20694.0
4,JB573087,12/30/2018 04:00:00 PM,073XX W LUNT AVE,THEFT (ALL TYPES),FROM BUILDING,16,41.0,2018,42.007986,-87.810471,"(42.007985889, -87.810471311)",NaN,60714.0,31208.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
252809,JF420379,9/16/2022 13:48,026XX N NARRAGANSETT AVE,THEFT (ALL TYPES),RETAIL THEFT,25,36.0,2022,41.928077,-87.785606,"(41.928076536, -87.785606186)",NaN,60707.0,41309.0
252810,JF420478,9/1/2022 5:00,005XX W SURF ST,OTHER MINOR CRIMES,HARASSMENT BY TELEPHONE,19,44.0,2022,41.934305,-87.641485,"(41.934304581, -87.641484982)",NaN,60657.0,72113.0
252811,JF420319,7/8/2022 0:00,114XX S PRAIRIE AVE,THEFT (ALL TYPES),FRAUD OR CONFIDENCE GAME,5,9.0,2022,41.685544,-87.616813,"(41.685543881, -87.616812541)",NaN,60628.0,64489.0
252812,JF420102,9/27/2022 11:00,023XX E 70TH ST,THEFT (ALL TYPES),OVER $500,3,5.0,2022,41.768068,-87.567453,"(41.768068052, -87.567452932)",NaN,60649.0,48973.0


In [23]:
# Rename the column for population and drop unused rows
cen_cleaned_df = cen_merge_df[["Case Number", "Date", "Block", "Primary Type", "Description", "District", "Ward",
                                     "Year", "Latitude", "Longitude", "Location", "Population_y", "Zipcode"]]
cen_cleaned_df = cen_cleaned_df.rename(columns = {"Population_y" : "Population"})
cen_cleaned_df

,Case Number,Date,Block,Primary Type,Description,District,Ward,Year,Latitude,Longitude,Location,Population,Zipcode
0,JC101958,12/31/2018 11:00:00 PM,100XX W OHARE ST,THEFT (ALL TYPES),CREDIT CARD FRAUD,16,41.0,2018,41.976290,-87.905227,"(41.976290414, -87.905227221)",20694.0,60106.0
1,JB574311,12/31/2018 09:30:00 PM,042XX W 31ST ST,ASSAULT & BATTERY,SIMPLE,10,22.0,2018,41.836806,-87.730955,"(41.836806313, -87.73095549)",84905.0,60804.0
2,JC101161,12/31/2018 09:00:00 PM,030XX S KOLIN AVE,MINOR PROPERTY CRIME,TO VEHICLE,10,22.0,2018,41.837771,-87.732880,"(41.837771398, -87.732880358)",84905.0,60804.0
3,JB574199,12/31/2018 06:20:00 PM,100XX W OHARE ST,THEFT (ALL TYPES),CREDIT CARD FRAUD,16,41.0,2018,41.976290,-87.905227,"(41.976290414, -87.905227221)",20694.0,60106.0
4,JB573087,12/30/2018 04:00:00 PM,073XX W LUNT AVE,THEFT (ALL TYPES),FROM BUILDING,16,41.0,2018,42.007986,-87.810471,"(42.007985889, -87.810471311)",31208.0,60714.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
252809,JF420379,9/16/2022 13:48,026XX N NARRAGANSETT AVE,THEFT (ALL TYPES),RETAIL THEFT,25,36.0,2022,41.928077,-87.785606,"(41.928076536, -87.785606186)",41309.0,60707.0
252810,JF420478,9/1/2022 5:00,005XX W SURF ST,OTHER MINOR CRIMES,HARASSMENT BY TELEPHONE,19,44.0,2022,41.934305,-87.641485,"(41.934304581, -87.641484982)",72113.0,60657.0
252811,JF420319,7/8/2022 0:00,114XX S PRAIRIE AVE,THEFT (ALL TYPES),FRAUD OR CONFIDENCE GAME,5,9.0,2022,41.685544,-87.616813,"(41.685543881, -87.616812541)",64489.0,60628.0
252812,JF420102,9/27/2022 11:00,023XX E 70TH ST,THEFT (ALL TYPES),OVER $500,3,5.0,2022,41.768068,-87.567453,"(41.768068052, -87.567452932)",48973.0,60649.0


In [24]:
# Concatenate cen_cleaned_df and crime_cleaned_df to get the final dataset
final_df = pd.concat([crime_cleaned_df, cen_cleaned_df])
final_df

,Case Number,Date,Block,Primary Type,Description,District,Ward,Year,Latitude,Longitude,Location,Population,Zipcode
0,JC104662,12/31/2018 11:59:00 PM,112XX S SACRAMENTO AVE,MINOR PROPERTY CRIME,TO VEHICLE,22,19.0,2018,41.689079,-87.696064,"(41.689078832, -87.696064026)",28804.0,60655.0
1,JC100043,12/31/2018 11:57:00 PM,084XX S SANGAMON ST,MINOR PROPERTY CRIME,TO PROPERTY,6,21.0,2018,41.740521,-87.647391,"(41.740520866, -87.647390719)",68096.0,60620.0
2,JC100006,12/31/2018 11:56:00 PM,018XX S ALLPORT ST,ASSAULT & BATTERY,AGG: HANDS/FIST/FEET NO/MINOR INJURY,12,25.0,2018,41.857068,-87.657625,"(41.857068095, -87.657625201)",79205.0,60608.0
3,JC100031,12/31/2018 11:55:00 PM,078XX S SANGAMON ST,ASSAULT & BATTERY,DOMESTIC BATTERY SIMPLE,6,17.0,2018,41.751914,-87.647717,"(41.75191443, -87.647716532)",68096.0,60620.0
4,JC100026,12/31/2018 11:49:00 PM,052XX W GLADYS AVE,ASSAULT & BATTERY,AGGRAVATED - HANDGUN,15,29.0,2018,41.875684,-87.760479,"(41.87568438, -87.760479356)",47712.0,60644.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
252809,JF420379,9/16/2022 13:48,026XX N NARRAGANSETT AVE,THEFT (ALL TYPES),RETAIL THEFT,25,36.0,2022,41.928077,-87.785606,"(41.928076536, -87.785606186)",41309.0,60707.0
252810,JF420478,9/1/2022 5:00,005XX W SURF ST,OTHER MINOR CRIMES,HARASSMENT BY TELEPHONE,19,44.0,2022,41.934305,-87.641485,"(41.934304581, -87.641484982)",72113.0,60657.0
252811,JF420319,7/8/2022 0:00,114XX S PRAIRIE AVE,THEFT (ALL TYPES),FRAUD OR CONFIDENCE GAME,5,9.0,2022,41.685544,-87.616813,"(41.685543881, -87.616812541)",64489.0,60628.0
252812,JF420102,9/27/2022 11:00,023XX E 70TH ST,THEFT (ALL TYPES),OVER $500,3,5.0,2022,41.768068,-87.567453,"(41.768068052, -87.567452932)",48973.0,60649.0


In [25]:
# Drop any remaining null rows
final_df = final_df.dropna(how="any")
final_df

,Case Number,Date,Block,Primary Type,Description,District,Ward,Year,Latitude,Longitude,Location,Population,Zipcode
0,JC104662,12/31/2018 11:59:00 PM,112XX S SACRAMENTO AVE,MINOR PROPERTY CRIME,TO VEHICLE,22,19.0,2018,41.689079,-87.696064,"(41.689078832, -87.696064026)",28804.0,60655.0
1,JC100043,12/31/2018 11:57:00 PM,084XX S SANGAMON ST,MINOR PROPERTY CRIME,TO PROPERTY,6,21.0,2018,41.740521,-87.647391,"(41.740520866, -87.647390719)",68096.0,60620.0
2,JC100006,12/31/2018 11:56:00 PM,018XX S ALLPORT ST,ASSAULT & BATTERY,AGG: HANDS/FIST/FEET NO/MINOR INJURY,12,25.0,2018,41.857068,-87.657625,"(41.857068095, -87.657625201)",79205.0,60608.0
3,JC100031,12/31/2018 11:55:00 PM,078XX S SANGAMON ST,ASSAULT & BATTERY,DOMESTIC BATTERY SIMPLE,6,17.0,2018,41.751914,-87.647717,"(41.75191443, -87.647716532)",68096.0,60620.0
4,JC100026,12/31/2018 11:49:00 PM,052XX W GLADYS AVE,ASSAULT & BATTERY,AGGRAVATED - HANDGUN,15,29.0,2018,41.875684,-87.760479,"(41.87568438, -87.760479356)",47712.0,60644.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
252809,JF420379,9/16/2022 13:48,026XX N NARRAGANSETT AVE,THEFT (ALL TYPES),RETAIL THEFT,25,36.0,2022,41.928077,-87.785606,"(41.928076536, -87.785606186)",41309.0,60707.0
252810,JF420478,9/1/2022 5:00,005XX W SURF ST,OTHER MINOR CRIMES,HARASSMENT BY TELEPHONE,19,44.0,2022,41.934305,-87.641485,"(41.934304581, -87.641484982)",72113.0,60657.0
252811,JF420319,7/8/2022 0:00,114XX S PRAIRIE AVE,THEFT (ALL TYPES),FRAUD OR CONFIDENCE GAME,5,9.0,2022,41.685544,-87.616813,"(41.685543881, -87.616812541)",64489.0,60628.0
252812,JF420102,9/27/2022 11:00,023XX E 70TH ST,THEFT (ALL TYPES),OVER $500,3,5.0,2022,41.768068,-87.567453,"(41.768068052, -87.567452932)",48973.0,60649.0


In [26]:
# Make sure zip codes are in Chicago
sort_zip = final_df["Zipcode"].sort_values().unique()
sort_zip

array([60007., 60018., 60068., 60106., 60601., 60602., 60605., 60606.,
       60607., 60608., 60609., 60610., 60611., 60612., 60613., 60614.,
       60615., 60616., 60617., 60618., 60619., 60620., 60621., 60622.,
       60623., 60624., 60625., 60626., 60628., 60629., 60630., 60631.,
       60632., 60633., 60634., 60636., 60637., 60638., 60639., 60640.,
       60641., 60642., 60643., 60644., 60645., 60646., 60647., 60649.,
       60651., 60652., 60653., 60654., 60655., 60656., 60657., 60659.,
       60660., 60706., 60707., 60714., 60804., 60827.])

In [27]:
# This was checked with the Name Census
# List of 67 ZIP codes in Chicago, Illinois. Name Census. (n.d.). https://namecensus.com/zip-codes/illinois/city/chicago/ 
non_chicago_zips = [
    46320,
    60077,
    60171,
    60176,
    60301,
    60302,
    60304,
    60406,
    60419,
    60453,
    60456, 
    60459, 
    60501,
    60534, 
    60712,
    60803,
    60805,
    65692]
         

In [28]:
# Remove the zip codes that are outside of Chicago
final_df = final_df[~final_df["Zipcode"].isin(non_chicago_zips)]
final_df


,Case Number,Date,Block,Primary Type,Description,District,Ward,Year,Latitude,Longitude,Location,Population,Zipcode
0,JC104662,12/31/2018 11:59:00 PM,112XX S SACRAMENTO AVE,MINOR PROPERTY CRIME,TO VEHICLE,22,19.0,2018,41.689079,-87.696064,"(41.689078832, -87.696064026)",28804.0,60655.0
1,JC100043,12/31/2018 11:57:00 PM,084XX S SANGAMON ST,MINOR PROPERTY CRIME,TO PROPERTY,6,21.0,2018,41.740521,-87.647391,"(41.740520866, -87.647390719)",68096.0,60620.0
2,JC100006,12/31/2018 11:56:00 PM,018XX S ALLPORT ST,ASSAULT & BATTERY,AGG: HANDS/FIST/FEET NO/MINOR INJURY,12,25.0,2018,41.857068,-87.657625,"(41.857068095, -87.657625201)",79205.0,60608.0
3,JC100031,12/31/2018 11:55:00 PM,078XX S SANGAMON ST,ASSAULT & BATTERY,DOMESTIC BATTERY SIMPLE,6,17.0,2018,41.751914,-87.647717,"(41.75191443, -87.647716532)",68096.0,60620.0
4,JC100026,12/31/2018 11:49:00 PM,052XX W GLADYS AVE,ASSAULT & BATTERY,AGGRAVATED - HANDGUN,15,29.0,2018,41.875684,-87.760479,"(41.87568438, -87.760479356)",47712.0,60644.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
252809,JF420379,9/16/2022 13:48,026XX N NARRAGANSETT AVE,THEFT (ALL TYPES),RETAIL THEFT,25,36.0,2022,41.928077,-87.785606,"(41.928076536, -87.785606186)",41309.0,60707.0
252810,JF420478,9/1/2022 5:00,005XX W SURF ST,OTHER MINOR CRIMES,HARASSMENT BY TELEPHONE,19,44.0,2022,41.934305,-87.641485,"(41.934304581, -87.641484982)",72113.0,60657.0
252811,JF420319,7/8/2022 0:00,114XX S PRAIRIE AVE,THEFT (ALL TYPES),FRAUD OR CONFIDENCE GAME,5,9.0,2022,41.685544,-87.616813,"(41.685543881, -87.616812541)",64489.0,60628.0
252812,JF420102,9/27/2022 11:00,023XX E 70TH ST,THEFT (ALL TYPES),OVER $500,3,5.0,2022,41.768068,-87.567453,"(41.768068052, -87.567452932)",48973.0,60649.0


In [29]:
final_df.to_csv("chicago_crimes_all_cleaned.csv", index = False)